# Kaggle Fraud Dataset  
source: [www.kaggle.com/datasets/fraud.csv](https://www.kaggle.com/datasets/vardhansiramdasu/fraudulent-transactions-prediction/data)

| Feature | Description |
|---------|-------------|
|**step**| maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).| 
|**type**| CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.| 
|**amount**| amount of the transaction in local currency.| 
|**nameOrig**| customer who started the transaction| 
|**oldbalanceOrg**| initial balance before the transaction| 
|**newbalanceOrig**| new balance after the transaction| 
|**nameDest**| customer who is the recipient of the transaction| 
|**oldbalanceDest**| initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants).| 
|**newbalanceDest**| new balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants).| 
|**isFraud**| This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.|  
|**isFlaggedFraud**| The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.| 

In [79]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, ticker as mticker

from sklearn import metrics 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression 

from feature_engine import encoding as ce

from xgboost import XGBClassifier 

import optuna
from optuna.storages import RDBStorage
# from optuna_dashboard import wsgi

storage = RDBStorage("sqlite:///db.sqlite3")
# application = wsgi(storage)

## Load data

In [17]:
df = pd.read_csv('Fraud.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [18]:
# Imbalance of target 
print(df['isFraud'].value_counts(dropna=False))
print(df['isFraud'].value_counts(normalize=True, dropna=False))

0    6354407
1       8213
Name: isFraud, dtype: int64
0    0.998709
1    0.001291
Name: isFraud, dtype: float64


In [19]:
df.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [20]:
df.groupby('type')['type'].count()

type
CASH_IN     1399284
CASH_OUT    2237500
DEBIT         41432
PAYMENT     2151495
TRANSFER     532909
Name: type, dtype: int64

In [11]:
df['nameOrig_tf'] = [i[0:2] for i in df['nameOrig']]
df.groupby('nameOrig_tf')['nameOrig_tf'].count()

nameOrig_tf
C1    3290968
C2     766936
C3     327938
C4     329176
C5     330009
C6     329312
C7     330560
C8     328926
C9     328795
Name: nameOrig_tf, dtype: int64

In [12]:
df['nameDest_tf'] = [i[0:2] for i in df['nameDest']]
df.groupby('nameDest_tf')['nameDest_tf'].count()

nameDest_tf
C1    2177198
C2     509697
C3     216675
C4     216903
C5     216484
C6     216051
C7     221434
C8     218371
C9     218312
M1    1113250
M2     258735
M3     111298
M4     111116
M5     111343
M6     110943
M7     111760
M8     111350
M9     111700
Name: nameDest_tf, dtype: int64

In [43]:
df = pd.read_csv('Fraud.csv')
X_train, X_test, y_train, y_test = train_test_split(df.drop(['isFraud', 'isFlaggedFraud'], axis=1),
                                                    df['isFraud'],
                                                    test_size=0.2,
                                                    stratify=df['isFraud'],
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5090096, 9), (1272524, 9), (5090096,), (1272524,))

In [44]:
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

0    0.998709
1    0.001291
Name: isFraud, dtype: float64
0    0.998709
1    0.001291
Name: isFraud, dtype: float64


## Set up preprocessing pipeline

In [64]:
def preprocessing(X):
    
    X['nameOrig'] = [i[0:2] for i in X['nameOrig']]
    X['nameDest'] = [i[0:2] for i in X['nameDest']]
    return X

In [46]:
def performance(X, y, model, title=None):

    # Predict probabilities on the test set
    y_probs = model.predict_proba(X)[:, 1]
    y_preds = model.predict(X)

    # Calculate evaluation metrics
    roc_auc = metrics.roc_auc_score(y, y_probs)
    precision = metrics.precision_score(y, y_preds)
    recall = metrics.recall_score(y, y_preds)
    f1 = metrics.f1_score(y, y_preds)
    aupr = metrics.average_precision_score(y, y_probs)

    if title is None:
        title = 'Performance metrics:'
    else:
        title = f'Performance metrics: {title}'
    print(title)
    print("ROC AUC:".ljust(10) + f"{roc_auc:.2%}".rjust(8))
    print("Precision:".ljust(10) + f"{precision:.2%}".rjust(8))
    print("Recall:".ljust(10) + f"{recall:.2%}".rjust(8))
    print("F1:".ljust(10) + f"{f1:.2%}".rjust(8))
    print("AUPRC:".ljust(10) + f"{aupr:.2%}".rjust(8))
    print()

In [47]:
encoder = ce.OrdinalEncoder(encoding_method='ordered', 
                            variables=['type', 'nameOrig', 'nameDest'])

In [50]:
X_train = preprocessing(X_train)
X_train = encoder.fit_transform(X_train, y_train)
X_train.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
292779,15,2,9914.74,4,44248.00,34333.26,7,0.00,0.00
499763,20,2,6854.53,6,0.00,0.00,7,0.00,0.00
2970411,231,3,361211.80,2,0.00,0.00,12,489745.16,850956.95
3137549,236,2,7083.51,6,0.00,0.00,7,0.00,0.00
1500682,143,0,218019.51,6,13045685.58,13263705.09,16,2438123.98,2220104.47


In [57]:
model = XGBClassifier(n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [58]:
performance(X_train, y_train, model, title='X_train')

Performance metrics: X_train
ROC AUC:   100.00%
Precision:  98.43%
Recall:     90.49%
F1:         94.29%
AUPRC:      98.86%



In [55]:
X_test = preprocessing(X_test)
X_test = encoder.transform(X_test)
X_test.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
4051353,300,4,890577.21,6,218.00,0.00,16,0.00,890577.21
5746321,399,0,97734.24,6,2096258.84,2193993.08,12,320136.00,222401.76
6361797,718,2,5907.41,6,315.00,0.00,7,0.00,0.00
2247309,186,3,187696.30,6,11057.00,0.00,12,1798095.21,1985791.51
4692207,331,3,82646.52,0,0.00,0.00,12,1047805.87,1130452.39


In [59]:
performance(X_test, y_test, model, title='X_test')

Performance metrics: X_test
ROC AUC:    99.99%
Precision:  95.94%
Recall:     84.84%
F1:         90.05%
AUPRC:      96.76%



In [61]:
model_lr = LogisticRegression(max_iter=1000, random_state=42)
model_lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [62]:
performance(X_train, y_train, model_lr, title='X_train')
performance(X_test, y_test, model_lr, title='X_test')

Performance metrics: X_train
ROC AUC:    86.04%
Precision:  79.30%
Recall:     44.44%
F1:         56.96%
AUPRC:      45.95%

Performance metrics: X_test
ROC AUC:    86.42%
Precision:  79.63%
Recall:     44.49%
F1:         57.09%
AUPRC:      46.27%



## Machine-learning pipeline & k-fold cross-validation

In [68]:
df = pd.read_csv('Fraud.csv')
X_train, X_test, y_train, y_test = train_test_split(df.drop(['isFraud', 'isFlaggedFraud'], axis=1),
                                                    df['isFraud'],
                                                    test_size=0.2,
                                                    stratify=df['isFraud'],
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5090096, 9), (1272524, 9), (5090096,), (1272524,))

In [73]:
fraud_pipeline = Pipeline([
    
    # transform Origname and Destname to less granular features
   ('orig_dest_transformation', FunctionTransformer(preprocessing)),

    # categorical encoding
    ('encoder_categorical',ce.OrdinalEncoder(encoding_method='ordered',
                                             variables=['type', 'nameOrig', 'nameDest'])),
    
    # Extreme Gradiant Boosting model    
    # ('model', XGBClassifier(n_jobs=-1, random_state=42))
])

In [70]:
X_train = fraud_pipeline.fit_transform(X_train, y_train)
X_train.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
292779,15,2,9914.74,4,44248.00,34333.26,7,0.00,0.00
499763,20,2,6854.53,6,0.00,0.00,7,0.00,0.00
2970411,231,3,361211.80,2,0.00,0.00,12,489745.16,850956.95
3137549,236,2,7083.51,6,0.00,0.00,7,0.00,0.00
1500682,143,0,218019.51,6,13045685.58,13263705.09,16,2438123.98,2220104.47


In [71]:
X_test = fraud_pipeline.transform(X_test)
X_test.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
4051353,300,4,890577.21,6,218.00,0.00,16,0.00,890577.21
5746321,399,0,97734.24,6,2096258.84,2193993.08,12,320136.00,222401.76
6361797,718,2,5907.41,6,315.00,0.00,7,0.00,0.00
2247309,186,3,187696.30,6,11057.00,0.00,12,1798095.21,1985791.51
4692207,331,3,82646.52,0,0.00,0.00,12,1047805.87,1130452.39


In [77]:
# weight will be used as a hyperparameter to offset the class imbalance
weight = int(y_train[y_train==0].shape[0] / y_train[y_train==1].shape[0])
weight

773

In [80]:
def objective(trial):
    params = {
        'n_estimators' : trial.suggest_int('n_estimators', 100, 500),
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.11),
        'max_depth' : trial.suggest_int('max_depth', 2, 10),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 1, weight)
    }

    model = XGBClassifier(**params,n_jobs=-1, random_state=42)

    # Calculate F1 score using cross-validation
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
    f1_mean = scores.mean()  # Take the mean F1 score

    return f1_mean

In [82]:
study = optuna.create_study(
    storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
    study_name="Kaggle_Fraud_1",
    direction='maximize',
    sampler=optuna.samplers.RandomSampler(seed=42),
    load_if_exists=True
)
study.optimize(objective, n_trials=200)
print(f"Best value: {study.best_value} (params: {study.best_params})")

[I 2024-03-09 20:54:53,652] A new study created in RDB with name: KDD_Cup_2004_1
[I 2024-03-09 20:57:20,169] Trial 0 finished with value: 0.7595636910061376 and parameters: {'n_estimators': 250, 'learning_rate': 0.10507143064099161, 'max_depth': 8, 'scale_pos_weight': 463.16434980011223}. Best is trial 0 with value: 0.7595636910061376.
[I 2024-03-09 20:58:10,637] Trial 1 finished with value: 0.04989737556533853 and parameters: {'n_estimators': 162, 'learning_rate': 0.025599452033620268, 'max_depth': 2, 'scale_pos_weight': 669.6879845382499}. Best is trial 0 with value: 0.7595636910061376.
[I 2024-03-09 20:59:42,429] Trial 2 finished with value: 0.12874733437154368 and parameters: {'n_estimators': 341, 'learning_rate': 0.08080725777960454, 'max_depth': 2, 'scale_pos_weight': 749.7704058690596}. Best is trial 0 with value: 0.7595636910061376.
[I 2024-03-09 21:01:54,247] Trial 3 finished with value: 0.3297701016719637 and parameters: {'n_estimators': 433, 'learning_rate': 0.03123391106782

Best value: 0.8854823732036611 (params: {'n_estimators': 477, 'learning_rate': 0.10539285770025873, 'max_depth': 10, 'scale_pos_weight': 286.76251659720305})


In [84]:
# best choice of provided hyperparameters
params = study.best_params
params

{'n_estimators': 477,
 'learning_rate': 0.10539285770025873,
 'max_depth': 10,
 'scale_pos_weight': 286.76251659720305}

In [85]:
model_xgb = XGBClassifier(**params, n_jobs=-1, random_state=42)
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.10539285770025873,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=477, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [86]:
performance(X_train, y_train, model_xgb, title='X_train')
performance(X_test, y_test, model_xgb, title='X_test')

Performance metrics: X_train
ROC AUC:   100.00%
Precision:  97.05%
Recall:    100.00%
F1:         98.50%
AUPRC:     100.00%

Performance metrics: X_test
ROC AUC:    99.98%
Precision:  85.64%
Recall:     92.21%
F1:         88.80%
AUPRC:      96.49%



In [87]:
# XGBoost: default parameters
performance(X_train, y_train, model, title='X_train')
performance(X_test, y_test, model, title='X_test')

Performance metrics: X_train
ROC AUC:   100.00%
Precision:  98.43%
Recall:     90.49%
F1:         94.29%
AUPRC:      98.86%

Performance metrics: X_test
ROC AUC:    99.99%
Precision:  95.94%
Recall:     84.84%
F1:         90.05%
AUPRC:      96.76%

